In [1]:
import fuzzingbook_utils

# Efficient Regression Testing with Mined UI Grammars

__GOAL:__ Mine a single grammar containing UI Elements, UI States, UI Interactions and Statement Coverage. Consume this grammar to efficiently generate tests.

## Roadmap

1. __DONE__ Explore an app using DroidMate _(maybe mutiple times?)_.
2. __ONGOING__ Extract a UI grammar containing only the UI elements which DM interacted with (Kotlin).
3. __DONE__ Use the fuzzingbook's GrammarCoverageFuzzer to consume the Grammar and generate input values (Python).
4. __DONE__ Extend the funzzingbook's grammar coverage tracking to consider both terminals and non-terminals (Python). 
5. _PENDING_ Associate the grammar elements with _covered statements_ (Kotlin).
6. _PENDING_ Consume expanded grammar to generatetests targeted to specific code locations (Kotlin).
7. _DONE_ Generate inputs to cover all grammar elements
8. _PENDING_ Generate inputs to cover all previously covered statements
9. _PENDING_ Generate inputs to cover specific statements

## Grammar definition:

The grammar contains two types of productions _<state>_ and _<transition>_, where:

### State (Non-Terminal <sX\>)

Represents a UI state. It contains all _UI elements_ which can be interacted with on the state, alongside their transitions. 
It is defined as 

```
<sX> ::= w1<t1>, ..., wN<tN>
```

Where:
* wX is a widget
* <tX\> is a transition

### Transition (Non-Terminal <tX\>)

Represents a UI state transition trigger by a UI element interaction. 

By design, is always associated to a UI element, but could be theoretically generalized for system events.
It is defined as

```
  <tX> ::= <s1>, ..., <sN>
```

Where _<sX\>_ is a state. 
A transition contains only UI state productions. 
It also contains _all UI states_ which can be reached by this transition.

### Widget/UI Element (Terminal wX)

A UI element represents a UI element interaction in the app. It is defined as 

```
wX ≡ widgetX.actionY[.dataZ]
```

Where:
* _widgetX_ is the unique UI element ID
* _actionY_ is the type of action performed
* _dataZ_ is the value used in the action, such as text input, if any. This parameter is optional.

This notation can simultaneously encode:
* UI elements
* Different action types on the same UI elements (click, long click, tick, etc, input text)
* Different input payloads (searched value)

### Terminate condition

An exploration can be over at any time. Thus, all _State_ productions can also be $\epsilon$.



# Concrete Example

## Notation

* State: Sequential numbering
* Widget: Sequential numbering (later mapped to UID)
* Action types: Click and Long Click
* Transition: <state.widget>. 
For example `s6.w7.Click` stands for: the widget `w7.Click` in the state `s6`.
This allows the same widget and interaction to have different transitions in different states.


## Grammar

In [2]:
simple_ui_grammar = {
'<empty>'             : 		[''],
'<s1.w0.Click>'       : 		['<empty>', '<s2>'],
'<s1>'                : 		['w0.Click <s1.w0.Click>'],
'<s2.w1.Click>'       : 		['<empty>', '<s3>'],
'<s2.w3.Click>'       : 		['<empty>', '<s4>'],
'<s2>'                : 		['w1.Click <s2.w1.Click>', 'w3.Click <s2.w3.Click>'],
'<s3.w2.Click>'       : 		['<empty>', '<s2>'],
'<s3>'                : 		['w2.Click <s3.w2.Click>'],
'<s4.w4.Click>'       : 		['<empty>', '<s4>'],
'<s4.w5.LongClick>'   : 		['<empty>', '<s5>'],
'<s4>'                : 		['w4.Click <s4.w4.Click>', 'w5.LongClick <s4.w5.LongClick>'],
'<s5.w6.Click>'       : 		['<empty>', '<s6>'],
'<s5>'                : 		['w6.Click <s5.w6.Click>'],
'<s6.w7.Click>'       : 		['<empty>', '<s6>'],
'<s6.w8.Click>'       : 		['<empty>', '<s6>'],
'<s6>'                : 		['w7.Click <s6.w7.Click>', 'w8.Click <s6.w8.Click>'],
'<start>'             : 		['<empty>', '<s1>']
}

# Fuzzing

## Using expansion coverage

With fuzzingbook's default implementation

In [3]:
from GrammarCoverageFuzzer import GrammarCoverageFuzzer

In [4]:
fuzzer = GrammarCoverageFuzzer(simple_ui_grammar, min_nonterminals=1)
max_exp = fuzzer.max_expansion_coverage(max_depth=len(simple_ui_grammar))
while len(max_exp - fuzzer.expansion_coverage()) > 0:
    print(fuzzer.fuzz())


w0.Click w1.Click w2.Click 
w0.Click 
w0.Click w3.Click 
w0.Click w3.Click w4.Click 
w0.Click w1.Click 
w0.Click w3.Click w5.LongClick 
w0.Click w3.Click w5.LongClick w6.Click w8.Click 
w0.Click w3.Click w4.Click w5.LongClick w6.Click 
w0.Click w1.Click w2.Click w3.Click w5.LongClick w6.Click w7.Click 
w0.Click w3.Click w5.LongClick w6.Click w7.Click w8.Click w8.Click w7.Click 


## Using Term Coverage (Terminals and non-Terminals)

Extension of fuzzingbook's implementation.

Greedy algorithm.

In [5]:
import re
from Grammars import START_SYMBOL, RE_NONTERMINAL, nonterminals
from GrammarCoverageFuzzer import GrammarCoverageFuzzer

### Extended Coverage measurement

Number of times each symbol has been seem (necessary as all states can finish with $\epsilon$)

In [6]:
class TerminalCoverageGrammar(GrammarCoverageFuzzer):
    def add_coverage(self, symbol, new_children):
        """Coverage computation: number of times each symbol has been seen"""
        if self.log and symbol not in self.covered_expansions.keys():
            print("Now covered:", symbol)

        if symbol not in self.covered_expansions.keys():
            self.covered_expansions[symbol] = 0

        curr_val = self.covered_expansions[symbol]
        self.covered_expansions[symbol] = curr_val + 1
    def reset_coverage(self):
        self.covered_expansions = dict()

    def expansion_coverage(self):
        return self.covered_expansions.keys()

### Obtain set of existing symbols

Compute all existing symbols (terminals and non terminals) in the grammar:

In [7]:
class TerminalCoverageGrammar(TerminalCoverageGrammar):
    def all_symbols_grammar(self):
        symbols = { START_SYMBOL }

        for symbol in self.grammar.keys():
            for expansion in self.grammar[symbol]:
                for expansion_segment in list(filter(lambda x: x != "", re.split(RE_NONTERMINAL, expansion))):
                    symbols.add(expansion_segment)

        return symbols


### Redefine non-covered elements

Expansions with non-covered elements must use the new data type:

In [8]:
class TerminalCoverageGrammar(TerminalCoverageGrammar):
    def _uncovered_children(self, possible_children):
        # Prefer uncovered expansions
        uncovered_children = []
        index_map = []
        for idx, child_list in enumerate(possible_children):
            for child_tuple in child_list:
                (child, grand_child) = child_tuple
                if child not in self.expansion_coverage() and child_list not in uncovered_children:
                    uncovered_children.append(child_list)
                    index_map.append(idx)

        return index_map, uncovered_children

### Determine minimally covered elements

The same UI elements may be available in multiple states. Thus we should prioretize elements which have been seen the least when all elements have already been covered.

In [9]:
class TerminalCoverageGrammar(TerminalCoverageGrammar):
    def _minimally_expanded(self, possible_children):
        symbol_count = dict()
        for idx, child_list in enumerate(possible_children):
            for child_tuple in child_list:
                (child, grand_child) = child_tuple

                assert(child in self.covered_expansions.keys())
                symbol_count[child] = self.covered_expansions[child]

        min_count = min(symbol_count.values())

        # Prefer least covered expansions
        uncovered_children = []
        index_map = []
        for idx, child_list in enumerate(possible_children):
            for child_tuple in child_list:
                (child, grand_child) = child_tuple
                if symbol_count[child] == min_count and child_list not in uncovered_children:
                    uncovered_children.append(child_list)
                    index_map.append(idx)

        return index_map, uncovered_children

### Add terminals to coverage

Add terminals to the list of covered grammar elements

In [10]:
class TerminalCoverageGrammar(TerminalCoverageGrammar):
    def process_chosen_children(self, chosen_children, expansion):
        """Add terminals to coverage."""
        for expansion_segment in list(filter(lambda x: x != "", re.split(RE_NONTERMINAL, expansion))):
            if expansion_segment not in nonterminals(expansion):
                self.add_coverage(expansion_segment, chosen_children)
        return chosen_children


### Choose node expansion

Update node expansion rules to guide expansion towards least explored elements

In [11]:
class TerminalCoverageGrammar(TerminalCoverageGrammar):
    def choose_node_expansion(self, node, possible_children):
        index_map, uncovered_children = self._uncovered_children(possible_children)

        if len(uncovered_children) == 0:
            # Search for least explored
            index_map, min_covered_children = self._minimally_expanded(possible_children)
            # All expansions covered the same amount of times - use superclass method
            index = self.choose_covered_node_expansion(node, min_covered_children)

            return index_map[index]

        # Select from uncovered nodes
        index = self.choose_uncovered_node_expansion(node, uncovered_children)

        return index_map[index]

### Running the Fuzzer

In [12]:
fuzzer = TerminalCoverageGrammar(simple_ui_grammar, min_nonterminals=1, log=False)
max_exp = fuzzer.all_symbols_grammar()
while len(max_exp - fuzzer.expansion_coverage()) > 0:
    print(fuzzer.fuzz())

w0.Click w1.Click 

w0.Click w3.Click w5.LongClick w6.Click w7.Click w8.Click w8.Click 
w0.Click w3.Click w4.Click w5.LongClick w6.Click 
w0.Click w1.Click w2.Click 


# Larger example (XKCD - 20 Actions)

In [13]:
xkcd_grammar = {
'<empty>'             : 		[''],
'<s01.w00.ClickEvent>' : 		['<empty>', '<s02>'],
'<s01.w03.ClickEvent>' : 		['<empty>', '<s04>'],
'<s01.w07.ClickEvent>' : 		['<empty>', '<s07>'],
'<s01>'               : 		['w00.ClickEvent <s01.w00.ClickEvent>', 'w02.LongClickEvent <start>', 'w03.ClickEvent <s01.w03.ClickEvent>', 'w06.LongClickEvent <start>', 'w07.ClickEvent <s01.w07.ClickEvent>'],
'<s02.w01.Tick>'      : 		['<empty>', '<s03>'],
'<s02>'               : 		['w01.Tick <s02.w01.Tick>'],
'<s03.PressBack>'     : 		['<empty>', '<s01>', '<s07>'],
'<s03>'               : 		['<s03.PressBack>'],
'<s04.w04.ClickEvent>' : 		['<empty>', '<s05>'],
'<s04>'               : 		['w04.ClickEvent <s04.w04.ClickEvent>'],
'<s05.w05.ClickEvent>' : 		['<empty>', '<s06>'],
'<s05>'               : 		['w05.ClickEvent <s05.w05.ClickEvent>'],
'<s06>'               : 		['<start>'],
'<s07.w00.ClickEvent>' : 		['<empty>', '<s09>'],
'<s07.w02.ClickEvent>' : 		['<empty>', '<s10>'],
'<s07.w06.ClickEvent>' : 		['<empty>', '<s08>'],
'<s07>'               : 		['<empty>', 'w00.ClickEvent <s07.w00.ClickEvent>', 'w02.ClickEvent <s07.w02.ClickEvent>', 'w03.LongClickEvent <s01.w07.ClickEvent>', 'w06.ClickEvent <s07.w06.ClickEvent>', 'w07.ClickEvent <s01.w07.ClickEvent>'],
'<s08>'               : 		['w08.ClickEvent <s01.w07.ClickEvent>'],
'<s09>'               : 		['w09.ClickEvent <s02.w01.Tick>'],
'<s10>'               : 		['w10.Click <s01.w07.ClickEvent>'],
'<start>'             : 		['<empty>', '<s01>']
}

In [14]:
fuzzer = TerminalCoverageGrammar(xkcd_grammar, min_nonterminals=1, log=False)
max_exp = fuzzer.all_symbols_grammar()
reached = set()
while len(max_exp - fuzzer.expansion_coverage()) > 0:
    inp = fuzzer.fuzz()
    reached.add(inp)
    
for x in reached:
    print(x)


w02.LongClickEvent 
w03.ClickEvent w04.ClickEvent w05.ClickEvent 
w06.LongClickEvent 
w00.ClickEvent w01.Tick w03.LongClickEvent w02.ClickEvent w10.Click w06.ClickEvent w08.ClickEvent w06.ClickEvent w08.ClickEvent 
w07.ClickEvent w00.ClickEvent w09.ClickEvent 


# Another Example (aMetro - 20 Actions)

In [15]:
ametro_grammar = {
    '<ClickEvent(s01.w00)>' : 		['<empty>', '<s02>'],
    '<ClickEvent(s01.w03)>' : 		['<empty>', '<s03>'],
    '<ClickEvent(s02.w08)>' : 		['<empty>', '<s05>'],
    '<ClickEvent(s02.w12)>' : 		['<empty>', '<s07>'],
    '<ClickEvent(s03.w04)>' : 		['<empty>', '<s04>'],
    '<ClickEvent(s05.w09)>' : 		['<empty>', '<s06>'],
    '<ClickEvent(s07.w13)>' : 		['<empty>', '<s08>'],
    '<ClickEvent(s08.w14)>' : 		['<empty>', '<s09>'],
    '<ClickEvent(s09.w15)>' : 		['<empty>', '<s10>'],
    '<empty>'             : 		[''],
    '<s01>'               : 		['ClickEvent(w00) <ClickEvent(s01.w00)>', 'ClickEvent(w03) <ClickEvent(s01.w03)>', 'LongClickEvent(w02) <start>'],
    '<s02>'               : 		['ClickEvent(w01) <start>', 'ClickEvent(w08) <ClickEvent(s02.w08)>', 'ClickEvent(w12) <ClickEvent(s02.w12)>'],
    '<s03>'               : 		['ClickEvent(w04) <ClickEvent(s03.w04)>', 'ClickEvent(w06) <start>', 'ClickEvent(w07) <ClickEvent(s01.w00)>'],
    '<s04>'               : 		['ClickEvent(w01) <ClickEvent(s01.w03)>', 'LongClickEvent(w05) <ClickEvent(s03.w04)>'],
    '<s05>'               : 		['ClickEvent(w09) <ClickEvent(s05.w09)>'],
    '<s06>'               : 		['ClickEvent(w11) <ClickEvent(s01.w00)>', 'LongClickEvent(w10) <ClickEvent(s05.w09)>'],
    '<s07>'               : 		['ClickEvent(w13) <ClickEvent(s07.w13)>'],
    '<s08>'               : 		['ClickEvent(w14) <ClickEvent(s08.w14)>'],
    '<s09>'               : 		['ClickEvent(w15) <ClickEvent(s09.w15)>'],
    '<s10>'               : 		['<empty>', 'ClickEvent(w16) <ClickEvent(s09.w15)>'],
    '<start>'             : 		['<empty>', '<s01>']
}

In [16]:
fuzzer = TerminalCoverageGrammar(ametro_grammar, min_nonterminals=1, log=False)
max_exp = fuzzer.all_symbols_grammar()
reached = set()
while len(max_exp - fuzzer.expansion_coverage()) > 0:
    inp = fuzzer.fuzz()
    reached.add(inp)
    
for x in reached:
    print(x)


ClickEvent(w00) ClickEvent(w08) ClickEvent(w09) ClickEvent(w11) ClickEvent(w08) ClickEvent(w09) LongClickEvent(w10) LongClickEvent(w10) ClickEvent(w11) ClickEvent(w01) ClickEvent(w03) ClickEvent(w04) LongClickEvent(w05) LongClickEvent(w05) LongClickEvent(w05) LongClickEvent(w05) ClickEvent(w01) ClickEvent(w07) ClickEvent(w12) ClickEvent(w13) ClickEvent(w14) ClickEvent(w15) ClickEvent(w16) ClickEvent(w16) ClickEvent(w16) ClickEvent(w16) 
ClickEvent(w03) 
LongClickEvent(w02) 
ClickEvent(w03) ClickEvent(w06) 
LongClickEvent(w02) ClickEvent(w00) ClickEvent(w08) ClickEvent(w09) LongClickEvent(w10) ClickEvent(w11) ClickEvent(w12) ClickEvent(w13) ClickEvent(w14) ClickEvent(w15) ClickEvent(w16) ClickEvent(w16) ClickEvent(w16) ClickEvent(w16) ClickEvent(w16) 
ClickEvent(w03) ClickEvent(w07) ClickEvent(w12) ClickEvent(w13) ClickEvent(w14) ClickEvent(w15) ClickEvent(w16) ClickEvent(w16) 
ClickEvent(w00) ClickEvent(w01) 
LongClickEvent(w02) ClickEvent(w00) ClickEvent(w01) 


# aMetro - 100 actions

In [17]:
test_grammar = {
'<Click(s16.w27)>'    : 		['<empty>', '<s27>'],
'<Click(s19.w27)>'    : 		['<empty>', '<s20>'],
'<Click(s25.w27)>'    : 		['<empty>', '<s26>'],
'<ClickEvent(s01.w00)>' : 		['<empty>', '<s02>', '<s16>'],
'<ClickEvent(s01.w03)>' : 		['<empty>', '<s03>'],
'<ClickEvent(s02.w08)>' : 		['<empty>', '<s05>'],
'<ClickEvent(s02.w10)>' : 		['<empty>', '<s07>'],
'<ClickEvent(s02.w12)>' : 		['<empty>', '<s07>', '<s13>'],
'<ClickEvent(s03.w04)>' : 		['<empty>', '<s04>'],
'<ClickEvent(s03.w07)>' : 		['<empty>', '<s02>'],
'<ClickEvent(s04.w01)>' : 		['<empty>', '<s03>', '<s27>'],
'<ClickEvent(s05.w09)>' : 		['<empty>', '<s06>'],
'<ClickEvent(s07.w02)>' : 		['<empty>', '<s08>'],
'<ClickEvent(s08.w13)>' : 		['<empty>', '<s09>'],
'<ClickEvent(s09.w14)>' : 		['<empty>', '<s10>'],
'<ClickEvent(s09.w16)>' : 		['<empty>', '<s11>'],
'<ClickEvent(s11.w17)>' : 		['<empty>', '<s12>'],
'<ClickEvent(s12.w18)>' : 		['<empty>', '<s13>'],
'<ClickEvent(s12.w25)>' : 		['<empty>', '<s18>'],
'<ClickEvent(s13.w19)>' : 		['<empty>', '<s14>'],
'<ClickEvent(s13.w26)>' : 		['<empty>', '<s28>'],
'<ClickEvent(s13.w35)>' : 		['<empty>', '<s23>'],
'<ClickEvent(s14.w20)>' : 		['<empty>', '<s15>'],
'<ClickEvent(s15.w21)>' : 		['<empty>', '<s00>'],
'<ClickEvent(s18.w14)>' : 		['<empty>', '<s24>'],
'<ClickEvent(s18.w26)>' : 		['<empty>', '<s19>'],
'<ClickEvent(s21.w02)>' : 		['<empty>', '<s30>'],
'<ClickEvent(s21.w03)>' : 		['<empty>', '<s22>', '<s27>'],
'<ClickEvent(s21.w28)>' : 		['<empty>', '<s21>'],
'<ClickEvent(s22.w30)>' : 		['<empty>', '<s22>'],
'<ClickEvent(s24.w36)>' : 		['<empty>', '<s25>'],
'<ClickEvent(s26.w11)>' : 		['<empty>', '<s16>'],
'<ClickEvent(s28.w24)>' : 		['<empty>', '<s29>'],
'<LongClickEvent(s16.w22)>' : 		['<empty>', '<s17>'],
'<PressBack(s00)>'    : 		['<s01>'],
'<PressBack(s20)>'    : 		['<s21>'],
'<TextInsert(s30.w38,lixxdwxhjctsalrmgb)>' : 		['<empty>', '<s31>'],
'<empty>'             : 		[''],
'<s00>'               : 		['<PressBack(s00)>'],
'<s01>'               : 		['ClickEvent(w00) <ClickEvent(s01.w00)>', 'ClickEvent(w03) <ClickEvent(s01.w03)>', 'LongClickEvent(w02) <start>'],
'<s02>'               : 		['ClickEvent(w01) <start>', 'ClickEvent(w08) <ClickEvent(s02.w08)>', 'ClickEvent(w10) <ClickEvent(s02.w10)>', 'ClickEvent(w12) <ClickEvent(s02.w12)>', 'LongClickEvent(w02) <ClickEvent(s03.w07)>', 'LongClickEvent(w10) <ClickEvent(s03.w07)>'],
'<s03>'               : 		['ClickEvent(w00) <ClickEvent(s03.w07)>', 'ClickEvent(w04) <ClickEvent(s03.w04)>', 'ClickEvent(w06) <start>', 'ClickEvent(w07) <ClickEvent(s03.w07)>'],
'<s04>'               : 		['ClickEvent(w01) <ClickEvent(s04.w01)>', 'LongClickEvent(w05) <ClickEvent(s03.w04)>'],
'<s05>'               : 		['ClickEvent(w09) <ClickEvent(s05.w09)>', 'ClickEvent(w34) <ClickEvent(s01.w00)>'],
'<s06>'               : 		['ClickEvent(w11) <ClickEvent(s03.w07)>', 'ClickEvent(w12) <ClickEvent(s12.w18)>', 'LongClickEvent(w10) <ClickEvent(s05.w09)>', 'LongClickEvent(w23) <ClickEvent(s05.w09)>'],
'<s07>'               : 		['ClickEvent(w01) <ClickEvent(s03.w07)>', 'ClickEvent(w02) <ClickEvent(s07.w02)>'],
'<s08>'               : 		['ClickEvent(w13) <ClickEvent(s08.w13)>'],
'<s09>'               : 		['ClickEvent(w14) <ClickEvent(s09.w14)>', 'ClickEvent(w16) <ClickEvent(s09.w16)>'],
'<s10>'               : 		['ClickEvent(w15) <ClickEvent(s08.w13)>'],
'<s11>'               : 		['ClickEvent(w17) <ClickEvent(s11.w17)>'],
'<s12>'               : 		['ClickEvent(w18) <ClickEvent(s12.w18)>', 'ClickEvent(w25) <ClickEvent(s12.w25)>'],
'<s13>'               : 		['ClickEvent(w19) <ClickEvent(s13.w19)>', 'ClickEvent(w24) <ClickEvent(s11.w17)>', 'ClickEvent(w26) <ClickEvent(s13.w26)>', 'ClickEvent(w35) <ClickEvent(s13.w35)>'],
'<s14>'               : 		['ClickEvent(w20) <ClickEvent(s14.w20)>'],
'<s15>'               : 		['ClickEvent(w21) <ClickEvent(s15.w21)>'],
'<s16>'               : 		['Click(w27) <Click(s16.w27)>', 'ClickEvent(w08) <ClickEvent(s02.w08)>', 'ClickEvent(w10) <ClickEvent(s12.w18)>', 'LongClickEvent(w22) <LongClickEvent(s16.w22)>'],
'<s17>'               : 		['ClickEvent(w23) <ClickEvent(s03.w07)>'],
'<s18>'               : 		['ClickEvent(w14) <ClickEvent(s18.w14)>', 'ClickEvent(w26) <ClickEvent(s18.w26)>'],
'<s19>'               : 		['Click(w27) <Click(s19.w27)>'],
'<s20>'               : 		['<PressBack(s20)>'],
'<s21>'               : 		['ClickEvent(w02) <ClickEvent(s21.w02)>', 'ClickEvent(w03) <ClickEvent(s21.w03)>', 'ClickEvent(w28) <ClickEvent(s21.w28)>', 'LongClickEvent(w02) <ClickEvent(s21.w28)>'],
'<s22>'               : 		['ClickEvent(w29) <ClickEvent(s21.w28)>', 'ClickEvent(w30) <ClickEvent(s22.w30)>', 'ClickEvent(w31) <ClickEvent(s21.w28)>', 'ClickEvent(w33) <ClickEvent(s22.w30)>', 'Tick(w32) <ClickEvent(s22.w30)>'],
'<s23>'               : 		['ClickEvent(w21) <ClickEvent(s12.w25)>'],
'<s24>'               : 		['ClickEvent(w36) <ClickEvent(s24.w36)>'],
'<s25>'               : 		['Click(w27) <Click(s25.w27)>'],
'<s26>'               : 		['ClickEvent(w11) <ClickEvent(s26.w11)>'],
'<s27>'               : 		['<empty>', 'ClickEvent(w04) <ClickEvent(s03.w04)>', 'ClickEvent(w07) <ClickEvent(s26.w11)>', 'ClickEvent(w29) <ClickEvent(s21.w28)>', 'ClickEvent(w31) <ClickEvent(s21.w28)>'],
'<s28>'               : 		['ClickEvent(w24) <ClickEvent(s28.w24)>'],
'<s29>'               : 		['ClickEvent(w37) <ClickEvent(s21.w28)>'],
'<s30>'               : 		['ClickEvent(w17) <ClickEvent(s21.w28)>', 'ClickEvent(w28) <ClickEvent(s21.w02)>', 'TextInsert(w38,bexyp) <TextInsert(s30.w38,lixxdwxhjctsalrmgb)>', 'TextInsert(w38,lixxdwxhjctsalrmgb) <TextInsert(s30.w38,lixxdwxhjctsalrmgb)>'],
'<s31>'               : 		['ClickEvent(w17) <ClickEvent(s21.w28)>', 'ClickEvent(w39) <ClickEvent(s21.w02)>'],
'<start>'             : 		['<empty>', '<s01>'],
}

In [18]:
fuzzer = TerminalCoverageGrammar(test_grammar, min_nonterminals=1, log=False)
max_exp = fuzzer.all_symbols_grammar()
reached = set()
while len(max_exp - fuzzer.expansion_coverage()) > 0:
    inp = fuzzer.fuzz()
    reached.add(inp)
    
for x in reached:
    print(x)


ClickEvent(w00) ClickEvent(w10) ClickEvent(w24) ClickEvent(w18) ClickEvent(w35) ClickEvent(w21) ClickEvent(w26) Click(w27) ClickEvent(w02) TextInsert(w38,bexyp) ClickEvent(w39) TextInsert(w38,lixxdwxhjctsalrmgb) ClickEvent(w17) ClickEvent(w03) ClickEvent(w30) Tick(w32) Tick(w32) ClickEvent(w33) ClickEvent(w30) ClickEvent(w31) ClickEvent(w28) LongClickEvent(w02) ClickEvent(w28) 
ClickEvent(w03) 
ClickEvent(w00) LongClickEvent(w22) ClickEvent(w23) LongClickEvent(w10) ClickEvent(w10) ClickEvent(w02) ClickEvent(w13) ClickEvent(w14) ClickEvent(w15) ClickEvent(w14) ClickEvent(w15) ClickEvent(w16) ClickEvent(w17) ClickEvent(w25) ClickEvent(w14) ClickEvent(w36) Click(w27) ClickEvent(w11) Click(w27) ClickEvent(w29) ClickEvent(w03) ClickEvent(w04) LongClickEvent(w05) ClickEvent(w01) ClickEvent(w06) ClickEvent(w03) ClickEvent(w07) LongClickEvent(w10) ClickEvent(w10) ClickEvent(w02) ClickEvent(w13) ClickEvent(w14) ClickEvent(w15) ClickEvent(w16) ClickEvent(w17) ClickEvent(w18) ClickEvent(w35) Cli

## Get all terminals from the grammar

In [19]:
list(filter(lambda x: not "<" in x, fuzzer.all_symbols_grammar()))

['ClickEvent(w39) ',
 'ClickEvent(w00) ',
 'ClickEvent(w29) ',
 'Click(w27) ',
 'ClickEvent(w14) ',
 'TextInsert(w38,bexyp) ',
 'ClickEvent(w24) ',
 'ClickEvent(w25) ',
 'LongClickEvent(w10) ',
 'LongClickEvent(w22) ',
 'ClickEvent(w31) ',
 'ClickEvent(w09) ',
 'ClickEvent(w21) ',
 'ClickEvent(w04) ',
 'ClickEvent(w07) ',
 'Tick(w32) ',
 'ClickEvent(w13) ',
 'ClickEvent(w03) ',
 'ClickEvent(w23) ',
 'ClickEvent(w36) ',
 'ClickEvent(w19) ',
 'ClickEvent(w26) ',
 'TextInsert(w38,lixxdwxhjctsalrmgb) ',
 'ClickEvent(w18) ',
 'ClickEvent(w10) ',
 'ClickEvent(w20) ',
 'LongClickEvent(w05) ',
 'ClickEvent(w15) ',
 'ClickEvent(w01) ',
 'ClickEvent(w11) ',
 'ClickEvent(w06) ',
 'LongClickEvent(w02) ',
 'ClickEvent(w34) ',
 'ClickEvent(w37) ',
 'ClickEvent(w08) ',
 'ClickEvent(w16) ',
 'ClickEvent(w30) ',
 'ClickEvent(w33) ',
 'ClickEvent(w12) ',
 'ClickEvent(w02) ',
 'ClickEvent(w35) ',
 'LongClickEvent(w23) ',
 'ClickEvent(w17) ',
 'ClickEvent(w28) ']

# Easy XKCD 500

In [20]:
test_grammar = {
'<Click(s10.w10)>'    : 		['<s01>', '<s02>', '<s06>'],
'<ClickEvent(s02.w02)>' : 		['<s10>', '<s30>'],
'<ClickEvent(s04.w04)>' : 		['<s02>', '<s05>'],
'<ClickEvent(s06.w02)>' : 		['<s06>', '<s10>', '<s24>'],
'<ClickEvent(s06.w03)>' : 		['<s04>', '<s28>'],
'<ClickEvent(s08.w02)>' : 		['<s08>', '<s27>'],
'<ClickEvent(s08.w09)>' : 		['<s09>', '<s21>'],
'<ClickEvent(s20.w02)>' : 		['<s08>', '<s29>'],
'<LongClickEvent(s08.w05)>' : 		['<s02>', '<s08>'],
'<LongClickEvent(s20.w05)>' : 		['<s08>', '<s20>'],
'<PressBack(s03)>'    : 		['<s01>', '<s06>', '<s07>'],
'<PressBack(s09)>'    : 		['<s00>', '<s06>'],
'<PressBack(s14)>'    : 		['<s20>', '<s31>'],
'<Tick(s02.w01)>'     : 		['<s03>', '<s13>'],
'<Tick(s04.w07)>'     : 		['<s01>', '<s06>'],
'<Tick(s08.w01)>'     : 		['<s03>', '<s06>', '<s21>'],
'<empty>'             : 		[''],
'<s00>'               : 		['<empty>'],
'<s01>'               : 		['<empty>', 'ClickEvent(w00) <s02>', 'ClickEvent(w02) <s10>', 'ClickEvent(w03) <s04>', 'ClickEvent(w05) <s06>', 'ClickEvent(w06) <s11>', 'LongClickEvent(w02) <s01>', 'LongClickEvent(w03) <s01>', 'LongClickEvent(w05) <s01>', 'LongClickEvent(w06) <s01>'],
'<s02>'               : 		['<empty>', 'ClickEvent(w02) <ClickEvent(s02.w02)>', 'ClickEvent(w03) <s04>', 'ClickEvent(w05) <s08>', 'ClickEvent(w06) <s31>', 'ClickEvent(w09) <s01>', 'LongClickEvent(w02) <s02>', 'LongClickEvent(w03) <s02>', 'LongClickEvent(w05) <s02>', 'LongClickEvent(w06) <s02>', 'Tick(w01) <Tick(s02.w01)>'],
'<s03>'               : 		['<empty>', 'PressBack(s03) <PressBack(s03)>'],
'<s04>'               : 		['<empty>', 'ClickEvent(w04) <ClickEvent(s04.w04)>', 'ClickEvent(w12) <Tick(s04.w07)>', 'Tick(w07) <Tick(s04.w07)>'],
'<s05>'               : 		['<empty>', 'PressBack(s05) <s01>'],
'<s06>'               : 		['<empty>', 'ClickEvent(w00) <s08>', 'ClickEvent(w02) <ClickEvent(s06.w02)>', 'ClickEvent(w03) <ClickEvent(s06.w03)>', 'ClickEvent(w05) <s06>', 'ClickEvent(w06) <s07>', 'LongClickEvent(w02) <s06>', 'LongClickEvent(w03) <s06>', 'LongClickEvent(w05) <s06>', 'LongClickEvent(w06) <s06>'],
'<s07>'               : 		['<empty>', 'ClickEvent(w02) <s06>', 'ClickEvent(w05) <s09>', 'ClickEvent(w08) <s06>', 'LongClickEvent(w02) <s07>', 'LongClickEvent(w03) <s07>', 'TextInsert(w11,hufojcmxghpteqrgf) <s12>'],
'<s08>'               : 		['<empty>', '<s00>', 'ClickEvent(w02) <ClickEvent(s08.w02)>', 'ClickEvent(w03) <ClickEvent(s06.w03)>', 'ClickEvent(w05) <s08>', 'ClickEvent(w06) <s20>', 'ClickEvent(w09) <ClickEvent(s08.w09)>', 'LongClickEvent(w02) <s08>', 'LongClickEvent(w03) <s08>', 'LongClickEvent(w05) <LongClickEvent(s08.w05)>', 'LongClickEvent(w06) <s08>', 'Tick(w01) <Tick(s08.w01)>'],
'<s09>'               : 		['<empty>', 'PressBack(s09) <PressBack(s09)>'],
'<s10>'               : 		['<empty>', 'Click(w10) <Click(s10.w10)>'],
'<s11>'               : 		['<empty>', 'ClickEvent(w02) <s16>', 'ClickEvent(w03) <s14>', 'ClickEvent(w08) <s01>', 'LongClickEvent(w05) <s11>', 'TextInsert(w11,xjtgdyxkr) <s12>'],
'<s12>'               : 		['<empty>', 'PressBack(s12) <s12>'],
'<s13>'               : 		['<empty>', 'ClickEvent(w14) <s01>', 'LongClickEvent(w13) <s13>'],
'<s14>'               : 		['<empty>', 'ClickEvent(w04) <s15>', 'PressBack(s14) <PressBack(s14)>'],
'<s15>'               : 		['<empty>', 'PressBack(s15) <s11>'],
'<s16>'               : 		['<empty>', 'ClickEvent(w15) <s17>'],
'<s17>'               : 		['<empty>', 'ClickEvent(w08) <s18>'],
'<s18>'               : 		['<empty>', 'ClickEvent(w00) <s19>'],
'<s19>'               : 		['<empty>', 'ClickEvent(w09) <s01>'],
'<s20>'               : 		['<empty>', 'ClickEvent(w02) <ClickEvent(s20.w02)>', 'ClickEvent(w03) <s14>', 'ClickEvent(w05) <s08>', 'ClickEvent(w09) <s03>', 'LongClickEvent(w02) <s20>', 'LongClickEvent(w05) <LongClickEvent(s20.w05)>', 'Tick(w01) <s07>'],
'<s21>'               : 		['<empty>', 'ClickEvent(w14) <s06>', 'ClickEvent(w16) <s22>', 'ClickEvent(w20) <s25>'],
'<s22>'               : 		['<empty>', 'ClickEvent(w14) <s06>', 'Tick(w17) <s23>'],
'<s23>'               : 		['<empty>', 'ClickEvent(w18) <s22>'],
'<s24>'               : 		['<empty>', 'ClickEvent(w00) <s08>', 'ClickEvent(w15) <s06>', 'ClickEvent(w19) <s06>'],
'<s25>'               : 		['<empty>', 'ClickEvent(w14) <s21>', 'ClickEvent(w16) <s26>', 'ClickEvent(w21) <s25>', 'Tick(w17) <s25>'],
'<s26>'               : 		['<empty>', 'Tick(w22) <s25>'],
'<s27>'               : 		['<empty>', 'ClickEvent(w02) <s08>', 'ClickEvent(w03) <s04>', 'ClickEvent(w15) <s08>', 'ClickEvent(w19) <s08>', 'LongClickEvent(w05) <s08>', 'LongClickEvent(w06) <s08>'],
'<s28>'               : 		['<empty>', 'ClickEvent(w12) <s06>', 'Tick(w07) <s06>'],
'<s29>'               : 		['<empty>', 'ClickEvent(w05) <s08>', 'ClickEvent(w15) <s20>', 'ClickEvent(w19) <s20>', 'LongClickEvent(w03) <s20>'],
'<s30>'               : 		['<empty>', 'ClickEvent(w15) <s02>', 'ClickEvent(w19) <s02>', 'LongClickEvent(w06) <s02>'],
'<s31>'               : 		['<empty>', 'ClickEvent(w02) <s02>', 'ClickEvent(w03) <s14>', 'ClickEvent(w05) <s08>', 'LongClickEvent(w02) <s31>', 'LongClickEvent(w05) <s31>'],
'<start>'             : 		['<empty>', '<s01>'],
}

In [21]:
for i in range(11):
    fuzzer = TerminalCoverageGrammar(test_grammar, min_nonterminals=1, log=False)
    max_exp = fuzzer.all_symbols_grammar()
    reached = set()
    while len(max_exp - fuzzer.expansion_coverage()) > 0:
        inp = fuzzer.fuzz()
        reached.add(inp)
    
    filename = '/Users/nataniel/Documents/saarland/repositories/test/droidgram/input/de.tap.easy_xkcd/inputs%02d.txt' % i
    print(filename)
    f = open(filename, 'w')
    
    for x in reached:
        f.write(x + "\n")
        #print(x)
        
    f.close()

/Users/nataniel/Documents/saarland/repositories/test/droidgram/input/de.tap.easy_xkcd/inputs00.txt
/Users/nataniel/Documents/saarland/repositories/test/droidgram/input/de.tap.easy_xkcd/inputs01.txt
/Users/nataniel/Documents/saarland/repositories/test/droidgram/input/de.tap.easy_xkcd/inputs02.txt
/Users/nataniel/Documents/saarland/repositories/test/droidgram/input/de.tap.easy_xkcd/inputs03.txt
/Users/nataniel/Documents/saarland/repositories/test/droidgram/input/de.tap.easy_xkcd/inputs04.txt
/Users/nataniel/Documents/saarland/repositories/test/droidgram/input/de.tap.easy_xkcd/inputs05.txt
/Users/nataniel/Documents/saarland/repositories/test/droidgram/input/de.tap.easy_xkcd/inputs06.txt
/Users/nataniel/Documents/saarland/repositories/test/droidgram/input/de.tap.easy_xkcd/inputs07.txt
/Users/nataniel/Documents/saarland/repositories/test/droidgram/input/de.tap.easy_xkcd/inputs08.txt
/Users/nataniel/Documents/saarland/repositories/test/droidgram/input/de.tap.easy_xkcd/inputs09.txt
/Users/nat

# Drinks

In [22]:
test_grammar = {
    '<ClickEvent(s01.w00)>' : 		['<s02>', '<s10>', '<s13>', '<s15>', '<s19>', '<s22>'],
'<ClickEvent(s02.w02)>' : 		['<s01>', '<s07>'],
'<ClickEvent(s04.w00)>' : 		['<s13>', '<s19>', '<s22>'],
'<ClickEvent(s05.w05)>' : 		['<s06>', '<s20>', '<s24>'],
'<ClickEvent(s07.w00)>' : 		['<s08>', '<s09>', '<s11>', '<s16>', '<s17>', '<s26>'],
'<ClickEvent(s11.w00)>' : 		['<s21>', '<s25>'],
'<ClickEvent(s17.w00)>' : 		['<s23>', '<s27>'],
'<ClickEvent(s19.w02)>' : 		['<s01>', '<s04>'],
'<PressBack(s03)>'    : 		['<s02>', '<s09>'],
'<PressBack(s06)>'    : 		['<s01>', '<s04>', '<s07>'],
'<PressBack(s14)>'    : 		['<s13>', '<s15>'],
'<PressBack(s20)>'    : 		['<s01>', '<s04>', '<s07>', '<s12>'],
'<empty>'             : 		[''],
'<s00>'               : 		['<empty>'],
'<s01>'               : 		['<empty>', 'ClickEvent(w00) <ClickEvent(s01.w00)>', 'ClickEvent(w03) <s04>', 'ClickEvent(w04) <s05>', 'ClickEvent(w06) <s01>', 'ClickEvent(w07) <s07>', 'LongClickEvent(w04) <s01>'],
'<s02>'               : 		['<empty>', 'ClickEvent(w01) <s03>', 'ClickEvent(w02) <ClickEvent(s02.w02)>'],
'<s03>'               : 		['<empty>', 'PressBack(s03) <PressBack(s03)>'],
'<s04>'               : 		['<empty>', 'ClickEvent(w00) <ClickEvent(s04.w00)>', 'ClickEvent(w04) <s05>', 'ClickEvent(w06) <s04>', 'ClickEvent(w07) <s07>', 'ClickEvent(w08) <s01>', 'LongClickEvent(w04) <s04>', 'TextInsert(w10,havokalutlssyaxa) <s12>', 'TextInsert(w10,hzbgizpsprwjwiymmxi) <s12>', 'TextInsert(w10,klvlbxpacesklbamhvcwhq) <s12>', 'TextInsert(w10,mdvlfxyqrs) <s12>', 'TextInsert(w10,mxghpteqrgfnzdjsjggwx) <s12>', 'TextInsert(w10,pxhacjxvqoh) <s12>', 'TextInsert(w10,repnhuagxqyganbb) <s12>', 'TextInsert(w10,snlzxif) <s12>', 'TextInsert(w10,ueksvynyimboq) <s12>', 'TextInsert(w10,vzumetfsaiegjuepkgoxx) <s12>'],
'<s05>'               : 		['<empty>', 'ClickEvent(w05) <ClickEvent(s05.w05)>'],
'<s06>'               : 		['<empty>', 'PressBack(s06) <PressBack(s06)>'],
'<s07>'               : 		['<empty>', '<s00>', 'ClickEvent(w00) <ClickEvent(s07.w00)>', 'ClickEvent(w04) <s05>', 'ClickEvent(w06) <s01>', 'ClickEvent(w07) <s07>', 'LongClickEvent(w04) <s07>'],
'<s08>'               : 		['<empty>', 'ClickEvent(w00) <s13>', 'ClickEvent(w02) <s07>'],
'<s09>'               : 		['<empty>', 'ClickEvent(w00) <s18>', 'ClickEvent(w01) <s03>', 'ClickEvent(w02) <s07>'],
'<s10>'               : 		['<empty>', 'ClickEvent(w02) <s01>', 'LongClickEvent(w09) <s10>'],
'<s11>'               : 		['<empty>', 'ClickEvent(w00) <ClickEvent(s11.w00)>', 'ClickEvent(w02) <s07>'],
'<s12>'               : 		['<empty>', 'ClickEvent(w04) <s05>', 'ClickEvent(w06) <s12>', 'ClickEvent(w07) <s07>', 'ClickEvent(w08) <s04>', 'LongClickEvent(w04) <s12>'],
'<s13>'               : 		['<empty>', 'ClickEvent(w02) <PressBack(s06)>', 'ClickEvent(w09) <s14>', 'LongClickEvent(w09) <s13>'],
'<s14>'               : 		['<empty>', 'PressBack(s14) <PressBack(s14)>'],
'<s15>'               : 		['<empty>', 'ClickEvent(w02) <ClickEvent(s02.w02)>', 'ClickEvent(w09) <s14>'],
'<s16>'               : 		['<empty>', 'ClickEvent(w00) <s15>', 'ClickEvent(w02) <s07>'],
'<s17>'               : 		['<empty>', 'ClickEvent(w00) <ClickEvent(s17.w00)>', 'ClickEvent(w02) <s07>'],
'<s18>'               : 		['<empty>', 'ClickEvent(w02) <s07>'],
'<s19>'               : 		['<empty>', 'ClickEvent(w02) <ClickEvent(s19.w02)>'],
'<s20>'               : 		['<empty>', 'PressBack(s20) <PressBack(s20)>'],
'<s21>'               : 		['<empty>', 'ClickEvent(w02) <s07>'],
'<s22>'               : 		['<empty>', 'ClickEvent(w02) <ClickEvent(s19.w02)>'],
'<s23>'               : 		['<empty>', 'ClickEvent(w02) <s07>'],
'<s24>'               : 		['<empty>', 'ClickEvent(w02) <PressBack(s06)>'],
'<s25>'               : 		['<empty>', 'ClickEvent(w02) <s07>'],
'<s26>'               : 		['<empty>', 'ClickEvent(w00) <s02>', 'ClickEvent(w02) <s07>'],
'<s27>'               : 		['<empty>', 'ClickEvent(w02) <s07>'],
'<s28>'               : 		['<empty>', 'ClickEvent(w03) <s04>'],
'<start>'             : 		['<empty>', '<s01>', '<s28>'],
}

In [23]:
for i in range(11):
    fuzzer = TerminalCoverageGrammar(test_grammar, min_nonterminals=1, log=False)
    max_exp = fuzzer.all_symbols_grammar()
    reached = set()
    while len(max_exp - fuzzer.expansion_coverage()) > 0:
        inp = fuzzer.fuzz()
        reached.add(inp)
    
    filename = '/Users/nataniel/Documents/saarland/repositories/test/droidgram/input/fr.masciulli.drinks/inputs%02d.txt' % i
    print(filename)
    f = open(filename, 'w')
    
    for x in reached:
        f.write(x + "\n")
        #print(x)
        
    f.close()

/Users/nataniel/Documents/saarland/repositories/test/droidgram/input/fr.masciulli.drinks/inputs00.txt
/Users/nataniel/Documents/saarland/repositories/test/droidgram/input/fr.masciulli.drinks/inputs01.txt
/Users/nataniel/Documents/saarland/repositories/test/droidgram/input/fr.masciulli.drinks/inputs02.txt
/Users/nataniel/Documents/saarland/repositories/test/droidgram/input/fr.masciulli.drinks/inputs03.txt
/Users/nataniel/Documents/saarland/repositories/test/droidgram/input/fr.masciulli.drinks/inputs04.txt
/Users/nataniel/Documents/saarland/repositories/test/droidgram/input/fr.masciulli.drinks/inputs05.txt
/Users/nataniel/Documents/saarland/repositories/test/droidgram/input/fr.masciulli.drinks/inputs06.txt
/Users/nataniel/Documents/saarland/repositories/test/droidgram/input/fr.masciulli.drinks/inputs07.txt
/Users/nataniel/Documents/saarland/repositories/test/droidgram/input/fr.masciulli.drinks/inputs08.txt
/Users/nataniel/Documents/saarland/repositories/test/droidgram/input/fr.masciulli.

# nerd.tuxmobil.fahrplan.congress - 500 actions